In [1]:
using Sundials

INFO: Precompiling module Sundials.


In [5]:
#
#  PFR with dispersion and rxn
#  Rxn: A + B -> C, B + C - > D
#


L = 10 # length
vz = 1 # velocity
D = 1.0e-4 # dispersion coefficient

# Reaction moddel parameters
k1 = 1.0
k2 = 1.0

# inlet concentrations
ca0 = 1.0
cb0 = 1.0
cc0 = 0.0
cd0 = 0.0

D_uL = D/(vz * L)

1.0e-5

In [59]:
function disprxn4(t, c, dc)

  D_uL = 1.0e-5
  nc = 4
  ngrid = 10000
  c0 = [1.0;1.0;0.0;0.0]

  dz = 1/(ngrid+1)

  Peloc = dz/D_uL

  alpha_lo = 1.0/dz + D_uL/dz^2
  alpha_mid = -1.0/dz - 2.0 * D_uL/dz^2
  alpha_hi = D_uL/dz^2

  dconc = zeros(ngrid, nc)
  conc = zeros(ngrid, nc)

  k = 0
  for i in 1:ngrid
    for j in 1:nc
      k = k + 1
      conc[i,j] = c[k]
    end
  end

  for i in 2:(ngrid-1)
    for j in 1:nc
      dconc[i,j] = alpha_lo * conc[i-1,j] + alpha_mid * conc[i,j] + alpha_hi * conc[i+1,j]
    end
  end

  for j in 1:nc
    dconc[1,j] = (alpha_mid + alpha_lo/(1.0+Peloc)) * conc[1,j] + alpha_lo * Peloc * c0[j]/(1 + Peloc) + alpha_hi * conc[2,j]
  end

  for j in 1:nc
    dconc[ngrid,j] = alpha_lo * conc[ngrid - 1,j] + (alpha_mid + alpha_hi) * conc[ngrid, j]
  end

  k = 0
  for i in 1:ngrid
    for j in 1:nc
      k = k + 1
      dc[k] = dconc[i,j]
    end
  end


end


disprxn4 (generic function with 1 method)

In [60]:
cinit = zeros(40000)
tspan = collect(0.0:0.1:3.0)

31-element Array{Float64,1}:
 0.0
 0.1
 0.2
 0.3
 0.4
 0.5
 0.6
 0.7
 0.8
 0.9
 1.0
 1.1
 1.2
 ⋮  
 1.9
 2.0
 2.1
 2.2
 2.3
 2.4
 2.5
 2.6
 2.7
 2.8
 2.9
 3.0

In [ ]:
res = Sundials.cvode(disprxn4, cinit, tspan)

In [50]:
using Plots

In [58]:
plot(tspan,res[:,397])